 ### Boundary Labels:
- The boundaries of the simulation box are denoted as follows:
  - **N**: North
  - **S**: South
  - **E**: East
  - **W**: West
  - **T**: Top
  - **B**: Bottom

### Voxel Space Orientation:
- **z-axis**: Represents the first dimension of the voxel space.
- **y-axis**: Represents the second dimension, with the 0th position being north.
- **x-axis**: Represents the third dimension, with the 0th position being to the west.

### Wind Direction and Boundary Conditions:
The simulation accounts for wind blowing horizontally at specific compass bearings, leading to 8 distinct cases for setting boundary conditions:

- **Cases 0 to 3** (Cardinal Directions): Wind at 0° (N), 90° (E), 180° (S), 270° (W). One boundary in either the xz or yz plane will have a specified velocity, either positive or negative, corresponding to the wind direction.

- **Cases 4 to 7** (Inter-Cardinal Directions):
  - **NE quadrant**: Positive y-velocity (vy), Negative x-velocity (vx)
  - **SE quadrant**: Negative y-velocity (vy), Negative x-velocity (vx)
  - **SW quadrant**: Negative y-velocity (vy), Positive x-velocity (vx)
  - **NW quadrant**: Positive y-velocity (vy), Positive x-velocity (vx)

### Cross Section and Land Boundaries:
- The landscape is represented within the voxel space, dividing the box's cross-section into areas of land and clear air.
- The interaction between the wind and the landscape introduces variations in boundary conditions across different parts of the box.

### Question on Boundary Conditions:
1. **Non-slip Conditions for Land Voxels**: Can all land voxels be treated as non-slip conditions to simulate the interaction with the wind accurately?

### Proposed Boundary Condition Implementation:
- **Wind Incident Borders**: For borders facing the direction of the incoming wind, apply constant velocity boundaries to simulate the wind entering the simulation space.
- **Do Nothing for the Ceiling**: The top boundary (ceiling) of the simulation box does not directly influence the wind simulation and may be treated as a 'do nothing' condition.
- **Remaining Borders and Air Spaces**: For parts of the box not defined as land or directly affected by the incoming wind, apply 'Do Not Care' or appropriate boundary conditions to simulate the natural behavior of air flow around and over the landscape.


# Generate a blank domain

In [ ]:
import numpy as np
import os
import sys

In [ ]:
# Assuming 'src' is in a directory at the same level as the current script's directory
# Adjust the path below according to your directory structure
xlb = "/mnt/c/Users/GGPC/Documents/GitHub/XLB"
__file__ = os.getcwd()
path_to_src = os.path.join(os.path.dirname(__file__), xlb)
sys.path.append(path_to_src)

In [ ]:
import matplotlib.pylab as plt
from src.models import BGKSim, KBCSim
from src.lattice import LatticeD3Q19, LatticeD3Q27
from src.boundary_conditions import *
import numpy as np
from src.utils import *
from jax import config
import os
#os.environ["XLA_FLAGS"] = '--xla_force_host_platform_device_count=8'
import jax
import scipy

In [ ]:
class Auckland_Harbour(KBCSim):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        #TODO
        # Test the following boundary conditions one at a time
"""
    def set_boundary_conditions(self):
        # Load the voxelised land and set to bounce back
        land_voxels = np.load("Akl_Voxel_Matrix.npy")
        land_indicies = np.argwhere(land_voxels)
    
        # Wall boundary conditions
        free_conditions = np.concatenate((self.boundingBoxIndices['bottom'], self.boundingBoxIndices['top'], self.boundingBoxIndices['front'], self.boundingBoxIndices['back'], self.boundingBoxIndices['right']))
        self.BCs.append(DoNothing(tuple(free_conditions.T), self.gridInfo, self.precisionPolicy))
    
        # Inlet boundary conditions
        inlet = self.boundingBoxIndices['left']
    
        # Convert inlet and land_indicies to sets of tuples for easy comparison
        inlet_set = set(map(tuple, inlet))
        land_indicies_set = set(map(tuple, land_indicies))
    
        # Find inlet voxels that are not land by set difference
        inlet_not_land = np.array(list(inlet_set - land_indicies_set))
    
        if inlet_not_land.size > 0:
            # Adjust the shape of inlet_not_land if necessary, depending on your simulation requirements
            # Apply conditions to the filtered inlet voxels
            rho_inlet = np.ones((inlet_not_land.shape[0], 1), dtype=self.precisionPolicy.compute_dtype)
            # Assuming the inlet velocity needs to be specified per voxel, adjust as necessary
            vel_inlet = np.zeros((inlet_not_land.shape[0], 3), dtype=self.precisionPolicy.compute_dtype)  # Assuming 3D velocity vectors
            # You would then need to apply these conditions appropriately within your simulation framework
            """


'\n    def set_boundary_conditions(self):\n        # Load the voxelised land and set to bounce back\n        land_voxels = np.load("Akl_Voxel_Matrix.npy")\n        land_indicies = np.argwhere(land_voxels)\n    \n        # Wall boundary conditions\n        free_conditions = np.concatenate((self.boundingBoxIndices[\'bottom\'], self.boundingBoxIndices[\'top\'], self.boundingBoxIndices[\'front\'], self.boundingBoxIndices[\'back\'], self.boundingBoxIndices[\'right\']))\n        self.BCs.append(DoNothing(tuple(free_conditions.T), self.gridInfo, self.precisionPolicy))\n    \n        # Inlet boundary conditions\n        inlet = self.boundingBoxIndices[\'left\']\n    \n        # Convert inlet and land_indicies to sets of tuples for easy comparison\n        inlet_set = set(map(tuple, inlet))\n        land_indicies_set = set(map(tuple, land_indicies))\n    \n        # Find inlet voxels that are not land by set difference\n        inlet_not_land = np.array(list(inlet_set - land_indicies_set))\n   

In [ ]:
precision = 'f32/f32'
lattice = LatticeD3Q27(precision)
nx = 601
ny = 351
nz = 251

Re = 50000.0
prescribed_vel = 0.05
clength = nx - 1

visc = prescribed_vel * clength / Re
omega = 1.0 / (3. * visc + 0.5)

os.system('rm -rf ./*.vtk && rm -rf ./*.png')

kwargs = {
        'lattice': lattice,
        'omega': omega,
        'nx': nx,
        'ny': ny,
        'nz': nz,
        'precision': precision,
        'io_rate': 100,
        'print_info_rate': 100,
        'return_fpost': True  # Need to retain fpost-collision for computation of lift and drag
    }
sim = Auckland_Harbour(**kwargs)
    #sim.run(200000)

**** Simulation Parameters for Auckland_Harbour ****
            Parameter | Value
--------------------------------------------------
                Omega | 1.9928258270227182
     Grid Points in X | 601
     Grid Points in Y | 351
     Grid Points in Z | 251
       Dimensionality | 3
     Precision Policy | f32/f32
         Lattice Type | D3Q27
      Checkpoint Rate | 0
 Checkpoint Directory | ./checkpoints
  Downsampling Factor | 1
      Print Info Rate | 100
             I/O Rate | 100
        Compute MLUPS | False
   Restore Checkpoint | False
              Backend | gpu
    Number of Devices | 1
Time to create the grid mask: 0.09856700897216797
Time to create the local masks and normal arrays: 1.1920928955078125e-06


In [ ]:
# Start measuring time for the voxelization process.
time_start = time()

# Determine the length of the car in lattice Boltzmann method (LBM) units. This script assumes the car length is one-fourth the size of the domain in the x-direction.
#NB Here we will just need to take our voxelised data and assign nx,ny and nz values?
car_length_lbm_unit = self.nx / 4

# Voxelizing the STL file. The `voxelize_stl` function converts the STL file into a voxel representation where the size of the voxel grid is determined by the car length in LBM units. It returns the voxelized object and the pitch (size of each voxel).
# is pitch used? 
car_voxelized, pitch = voxelize_stl(stl_filename, car_length_lbm_unit)

# Access the matrix representing the voxelized car. This matrix contains boolean values indicating whether a voxel is part of the car (True) or not (False).
#NB Adust the previous code to have boolean values, not integers
car_matrix = car_voxelized.matrix

# Print the time taken to voxelized the car and the pitch (voxel size) used.
print('Voxelization time for pitch={}: {} seconds'.format(pitch, time() - time_start))
print("Car matrix shape: ", car_matrix.shape)

# Calculate the area (in terms of voxels) occupied by the car by multiplying the dimensions of the car matrix, excluding the length dimension.
# NB this is actually the volume! 
self.car_area = np.prod(car_matrix.shape[1:])

# Calculate the amount of shift needed to place the car correctly within the simulation domain. This involves calculating the difference between the domain size and the car matrix size and then adjusting the position accordingly.
# OK - we don't need to do this
tx, ty, tz = np.array([nx, ny, nz]) - car_matrix.shape
shift = [tx//4, ty//2, 0]

# Find the indices of voxels that are part of the car and apply the shift to position the car within the simulation domain.
# NB modify this so that we are using our voxelised land
car_indices = np.argwhere(car_matrix) + shift

# Add a bounce-back boundary condition for the car's surface. This condition simulates a solid, immovable object within the fluid flow.
# NB ditto, we should just add the modified value calculated earlier
self.BCs.append(BounceBackHalfway(tuple(car_indices.T), self.gridInfo, self.precisionPolicy))

# Concatenate the indices of the bounding box's walls (bottom, top, front, back) to create a solid boundary around the simulation domain.
# NB This should be similar, but in this case the self bounding box where there is no wind indcident should be doNothing conditions
wall = np.concatenate((self.boundingBoxIndices['bottom'], self.boundingBoxIndices['top'], self.boundingBoxIndices['front'], self.boundingBoxIndices['back']))

# Add a bounce-back boundary condition for these walls, treating them as solid, immovable barriers.
# NB not present in this case
self.BCs.append(BounceBack(tuple(wall.T), self.gridInfo, self.precisionPolicy))

# Define a "do nothing" boundary condition for the right boundary, typically used to simulate an open or outflow boundary.
doNothing = self.boundingBoxIndices['right']
self.BCs.append(DoNothing(tuple(doNothing.T), self.gridInfo, self.precisionPolicy))
self.BCs[-1].implementationStep = 'PostCollision'
# The commented-out code below would set up a Zou/He pressure boundary condition on the right boundary, but it's not used in this snippet.

# Set up an inlet boundary condition on the left boundary with a prescribed velocity. This simulates fluid entering the simulation domain.
inlet = self.boundingBoxIndices['left']
rho_inlet = np.ones((inlet.shape[0], 1), dtype=self.precisionPolicy.compute_dtype) # Density at the inlet.
vel_inlet = np.zeros(inlet.shape, dtype=self.precisionPolicy.compute_dtype) # Velocity at the inlet, initialized to zero.

# Prescribe the velocity in the x-direction for the inlet boundary condition.
vel_inlet[:, 0] = prescribed_vel
self.BCs.append(EquilibriumBC(tuple(inlet.T), self.gridInfo, self.precisionPolicy, rho_inlet, vel_inlet))